<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/neuralnet_anatomie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Neuronale Netze
# ---------------

# Im Folgenden betrachten wir ein einfaches Beispiel eines Neuronalen Netzes -
# keine kompexere Variante wie z.B. rekurrente Netze (mit Gewichten innerhalb
# einer Schicht oder gar zu vorherigen Schichten), und keine Deep Learning-Netze
# (mit mehr als einer Zwischenschicht). Des Weiteren verzichten wir auf die
# Verwendung eines sog. Bias und einer sog. Lernarte, die normalerweise
# ebenfalls zu Neuronalen Netzen gehören.
# Die Vorgehensweise eines Neuronalen Netzes ist - je komplexer es wird -
# immer noch weniger vom Nutzer nachvollziehbar, bzw. ist schlicht zur Laufzeit
# eine "black box". Wir fokussieren uns daher auf ein ganz einfaches Beispiel,
# um eine Idee davon zu bekommen, was in einem Neuronalen Netz passiert.
# Wir werden keinen vorgefertigten NN-Algorithmus zur Validierung unseres
# eigenen Codes verwenden - ein so "einfaches" Beispiel ist damit kaum zu
# replizieren, und diese Beispiel ist ohnehin bereits komplex genug.

In [ ]:
# Wir wollen nun ein neuronales Netz mit folgender Struktur erstellen:
# - Inputschicht: 2 Neuronen
# - Zwischenschicht: 3 Neuronen
# - Outputschicht: 1 Neuron
# Es soll jedes Neuron mit jedem Neuron mittels eines Gewichtes verbunden sein.
# Die Gewichte werden ursprünglich mit random-Zahlen initialisiert.
# Wir simulieren ein Training, bestehend aus nur EINER Abfolge von Feedforward
# und Backpropagation, und wir trainieren einen Datensatz mit nur EINER Zeile.
# Diese Zeile besteht aus den Werten 1 und 0 als Input, und 0 als Label.
# Die Aktivierungsfunktion der Neuronen soll die Sigmoid-Funktion sein.

In [1]:
# Hierfür brauchen wir grundsätzlich wieder den Klassiker numpy,
# sowie das Paket random, um zufällige initiale Gewicht belegen zu können.
import numpy as np
import random

In [2]:
# Unsere Aktivierungsfunktion (also die Funktion, die das Eingangssignal eines
# Neurons zum tatsächlichen Wert des Neurons umwandelt) ist die Sigmoidfunktion:
# f(z)=1/(1+e^(-z)). z ist hierbei das Eingangssignal.

def sigmoid(X):
    return 1/(1 + np.exp(-X))

# Für die Bestimmung der backpropagation-Gewichte brauchen wir u.a. die
# partielle Ableitung der Sigmoid-Funktion (Herleitung ist hier nicht nötig).
def ableitung_sigmoid(X):
    return sigmoid(X) * (1 - sigmoid(X))

In [25]:
# Bauen wir zunächst das Neuronale Netz selbst, als eine Klasse NeuronalesNetz,
# aus der wir später ein Objekt dieser Klasse instanziieren können.
class NeuronalesNetz:

  # Konstruktor plus Attribute der Klasse
  def __init__(self, X, Y):
    # Input-Daten in der Inputschicht
    self.input = X
    # Output-Daten
    self.output = Y
    # Gewichte von der Inputschicht zur Zwischenschicht
    # (das sind 3 Inputschicht-Neuronen mal 4 Zwischenschichten-Neuronen =
    # 12 Gewichte)
    self.gewichte_input_zwischen = np.random.random_sample((self.input.shape[1],4))
    # Gewicht von der Zwischenschicht zur Outputschicht
    # (das sind 4 Zwischenschichten-Neuronen mal 1 Outputschicht-Neuron =
    # 4 Gewichte)
    self.gewichte_zwischen_output = np.random.random_sample((4,1))
    # Vorhersage des Neuronalen Netzes in der Outputschicht,
    # zunächst mit Nullern befüllt
    self.output_vorhersage = np.zeros(self.output.shape)

  # Nun müssen wir den feedforward-Prozess als Funktionen programmieren, sprich
  # die Berechnung der Signale z:

  def feedforward(self):
    # Die Werte der 4 Zwischenschicht-Neuronen werden berechnet mit
    # sigmoid(z)=1/(1+e^-z), mit z=W*x, wobei W die Gewichte von der Input-
    # zur Zwischenschicht sind, und x die Inputdaten
    self.werte_zwischen = sigmoid(np.dot(self.input, self.gewichte_input_zwischen))
    # Der Wert des Output-Neurons wird ebenfalls berechnet mit
    # sigmoid(z)=1/(1+e^-z), mit z=W*x, wobei W diesmal Gewichte von der
    # Zwischen- zur Outputschicht sind, und x die zuvor berechneten Werte der
    # 4 Zwischenschicht-Neuronen
    self.output_vorhersage = sigmoid(np.dot(self.werte_zwischen, self.gewichte_zwischen_output))

  # Nun müssen wir den backpropagation-Prozess als Funktion programmieren,
  # sprich die Berechnung der Veränderung der Gewichte, sodass der Verlust
  # minimal wird
  def backpropagation(self):
    # Die Anpassung der 4 Gewichte von der Zwischen- zur Outputschicht haben
    # wir explizit von Hand berechnet: Die Ableitung der Verlustfunktion nach
    # den Gewichten ist das Produkt aus drei einzelnen partiellen Ableitungen:
    # - 1. die Ableitung der Verlustfunktion nach dem vorhergesagten Output
    #      (2*(self.output - self.output_vorhersage)),
    # - 2. die Ableitung des vorhergesagten Outputs nach dem im Output-Neuron
    #      ankommenden Signal - also nichts anderes als die Ableitung der
    #      Sigmoid-Funktion (ableitung_sigmoid(self.output)), und
    # - 3. die Ableitung des im Output-Neuron ankommenden Signals nach den
    #      Gewichten von der Zwischenschicht zur Outputschicht
    #      (self.werte_zwischen.T):
    # Die Anpassung der 4 Gewichter von der Zwischen- zur Outputschicht
    # geschieht folgendermaßen:
    ableitung_gewichte_zwischen_output = np.dot(self.werte_zwischen.T, (-2*(self.output - self.output_vorhersage) * ableitung_sigmoid(self.output_vorhersage)))
    # Die Anpassung der 12 Gewichte von der Input- zur Zwischenschicht wird auf
    # die selbe Weise berechnet, haben wir aber aus Zeitgründen nicht
    # explizit gemacht:
    ableitung_gewichte_input_zwischen = np.dot(self.input.T, (np.dot(-2*(self.output - self.output_vorhersage) * ableitung_sigmoid(self.output_vorhersage), self.gewichte_zwischen_output.T) * ableitung_sigmoid(self.werte_zwischen)))
    # Nun müssen die Gewichte überschrieben werden: Die neuen Gewichte sind die
    # alten Gewichte plus der Verändung der Gewichte, wie sie genau durch die
    # Ableitung repräsentiert wird.
    self.gewichte_zwischen_output = self.gewichte_zwischen_output - ableitung_gewichte_zwischen_output
    self.gewichte_input_zwischen = self.gewichte_input_zwischen - ableitung_gewichte_input_zwischen


In [28]:
# Um die Ergebnisse wieder replizierbar zu machen, verwenden wir sog.
# Pseudo-Zufallszahlen, d.h. es werden bei Euch die selben Zufallszahlen gesetzt
# wie bei mir, sofern Ihr in der Klammer die selbe Zahl habt wie ich -
# welche Zahl das ist, spielt keine Rolle.
np.random.seed(1)

# Nun erzeugen wir Daten. Der Einfachheit halber nur einen Datensatz mit drei
# Input-Werten (für jedes Neuron in der Inputschicht einen), und einen
# Output-Wert.
if __name__ == "__main__":
  X = np.array([[0,0,1]])
  Y = np.array([[0]])

# Nun erzeugen wir ein Objekt der Klasse Neuonales Netzwerk:
  NN = NeuronalesNetz(X,Y)

# Nun machen wir erstmal lediglich eine Iteration der for-Schleife, um unsere
# von-Hand-Berechnungen zu verifizieren. Ihr könnt später gerne range()
# auf 1.000 oder höher stellen, um zu sehen, dass die Vorhersage des Y-Werts 0
# immer genauer wird!

# Für die von-Hand-Berechnung können wir uns hier (also nach der Erzeugung
# des Objekts, aber noch vor feedforward und backpropagation) die initialen,
# pseudo-zufällig gewählten Gewichte holen. Davor zwingen wir Python noch zur
# normalen Formatierung von Kommazahlen:
  np.set_printoptions(suppress=True)
  print(NN.gewichte_input_zwischen)
  print(NN.gewichte_zwischen_output)
  for i in range(10):
    NN.feedforward()
    NN.backpropagation()

# Zur Sicherstellung können die von Hand gerechneten Werte der
# Zwischenschicht-Neuronen hiermit überprüft werden:
  print(NN.werte_zwischen)

# Ebenso kann überprüft werden, ob die neuen Gewichte von der Zwischen- zur
# Outputschicht richtig berechnet wurden:
  print(NN.gewichte_zwischen_output)

# Nun können wir uns danach die Vorhersage ausgeben. Diese ist ca. 0,7575
# (also noch weit weg von der tatsächlichen 0, was nicht überraschend ist,
# weil der erste Durchlauf noch mit zufälligen, nicht-verbesserten Gewichten
# durchgeführt wurde). Hier kann ebenfalls überprüft werden, ob die von-Hand-
# Berechnung der Vorhersage des Neuronalen Netzes richtig ist:
  print(NN.output_vorhersage)

[[0.417022   0.72032449 0.00011437 0.30233257]
 [0.14675589 0.09233859 0.18626021 0.34556073]
 [0.39676747 0.53881673 0.41919451 0.6852195 ]]
[[0.20445225]
 [0.87811744]
 [0.02738759]
 [0.67046751]]
[[0.62133829 0.59280913 0.64341152 0.64952056]]
[[-0.89204667]
 [-0.22740906]
 [-1.09313778]
 [-0.51531432]]
[[0.16862495]]
